## Pre-Processing


In [5]:
!pip3 install nltk
!pip3 install gensim
!pip3 install pyLDAvis

     |████████████████████████████████| 1.5 MB 2.3 MB/s eta 0:00:01     |████████████▊                   | 593 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 136 kB/s eta 0:00:01
     |████████████████████████████████| 764 kB 184 kB/s eta 0:00:01
     |████████████████████████████████| 24.1 MB 28 kB/s  eta 0:00:01    |██████▋                         | 5.0 MB 2.5 MB/s eta 0:00:08     |██████████████                  | 10.5 MB 1.4 MB/s eta 0:00:10     |█████████████████████▎          | 16.0 MB 2.0 MB/s eta 0:00:05
     |████████████████████████████████| 58 kB 816 kB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 1.1 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 829 kB 993 kB/s eta 0:00:01
     |████████████████████████████████| 381 kB 1.1 MB/s eta 0:00:01
  Created wheel for

In [1]:
import pickle
import pandas as pd
import requests
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk

In [ ]:
nltk.download('popular')

In [57]:
df = pd.read_csv("../data/raw/allitems.csv", encoding='iso8859_15').drop(columns = ['Unnamed: 0'])
reserved = df['Description'][227237]
df

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Name,Status,Description,References,Phase,Votes,Comments
0,CVE-1999-0001,Candidate,ip_input.c in BSD-derived TCP/IP implementatio...,BUGTRAQ:19981223 Re: CERT Advisory CA-98.13 - ...,Modified (20051217),"MODIFY(1) Frech | NOOP(2) Northcutt, W...",Christey> A Bugtraq posting indicates that the...
1,CVE-1999-0002,Entry,Buffer overflow in NFS mountd gives root acces...,BID:121 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
2,CVE-1999-0003,Entry,Execute commands as root via buffer overflow i...,BID:122 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
3,CVE-1999-0004,Candidate,"MIME buffer overflow in email clients, e.g. So...",CERT:CA-98.10.mime_buffer_overflows | MS:M...,Modified (19990621),"ACCEPT(8) Baker, Cole, Collins, Dik, Landfi...","Frech> Extremely minor, but I believe e-mail i..."
4,CVE-1999-0005,Entry,Arbitrary command execution via IMAP buffer ov...,BID:130 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
227233,CVE-2022-24031,Candidate,** RESERVED ** This candidate has been reserve...,NaN,Assigned (20220126),None (candidate not yet proposed),NaN
227234,CVE-2022-24032,Candidate,** RESERVED ** This candidate has been reserve...,NaN,Assigned (20220126),None (candidate not yet proposed),NaN
227235,CVE-2022-24033,Candidate,** RESERVED ** This candidate has been reserve...,NaN,Assigned (20220126),None (candidate not yet proposed),NaN
227236,CVE-2022-24034,Candidate,** RESERVED ** This candidate has been reserve...,NaN,Assigned (20220126),None (candidate not yet proposed),NaN


In [67]:
df = df[~df.Description.str.startswith('**')]
desc = df['Description']

In [68]:
desc[1:20]

1     Buffer overflow in NFS mountd gives root acces...
2     Execute commands as root via buffer overflow i...
3     MIME buffer overflow in email clients, e.g. So...
4     Arbitrary command execution via IMAP buffer ov...
5     Buffer overflow in POP servers based on BSD/Qu...
6     Information from SSL-encrypted sessions via PK...
7     Buffer overflow in NIS+, in Sun's rpc.nisd pro...
8     Inverse query buffer overflow in BIND 4.9 and ...
9     Denial of Service vulnerability in BIND 8 Rele...
10    Denial of Service vulnerabilities in BIND 4.9 ...
11    Some web servers under Microsoft Windows allow...
12    Stolen credentials from SSH clients via ssh-ag...
13    Unauthorized privileged access or denial of se...
14                       Teardrop IP denial of service.
15                           Land IP denial of service.
16    FTP servers can allow an attacker to connect t...
17     Buffer overflow in statd allows root privileges.
18    Delete or create a file via rpc.statd, due

Remove Numbers

In [95]:
from string import digits
desc1 = ['']
remove_digits = str.maketrans("","",digits)
for line in desc:
     desc1.append(line.translate(remove_digits))

Remove Punctuation

Lemmitisation

Remove Stop words